In [101]:
import pandas as pd
import numpy as np
import os

In [102]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/SALT/Dashboards/')

In [209]:
df = pd.read_pickle('./CBAScoredFiles/eng.pkl')

In [210]:
drop = ['NQ16',
 'NQ17',
 'NQ18',
 'NQ19',
 'NQ20',
 'Score_NQ16',
 'Score_NQ17',
 'Score_NQ18',
 'Score_NQ19',
 'Score_NQ20']

In [211]:
df.drop(drop, axis = 1 , inplace = True)

In [212]:
unwanted = ('student_id','disecode','student_name','class','section','roll_no','unique_id','gender','social_category','medium_id'
            ,'Language_paperset','Language_jumble_set','Language_present_absent','LanguageTotal','LanguagePercentage_old','LanguagePercentage','Language_cheating_drop'
            ,'Maths_paperset','Maths_jumble_set','Maths_present_absent','MathsTotal','MathsPercentage_old','MathsPercentage','Maths_cheating_drop'
            ,'EVS_paperset','EVS_jumble_set','EVS_present_absent','EVSTotal','EVSPercentage_old','EVSPercentage','EVS_cheating_drop'
            ,'Science_paperset','Science_jumble_set','Science_present_absent','ScienceTotal','SciencePercentage_old','SciencePercentage','Science_cheating_drop'
            ,'SS_paperset','SS_jumble_set','SS_present_absent','SSTotal','SSPercentage_old','SSPercentage','SS_cheating_drop'
            ,'English_paperset','English_jumble_set','English_present_absent','EnglishTotal','EnglishPercentage_old','EnglishPercentage','English_cheating_drop'
           )

In [213]:
var_list = list(df)

In [214]:
#EQ = EVS, MQ = Maths, LQ = Language (Telugu), SQ = Science, OQ = Social Science
prefixes = ('EQ','MQ','LQ','SQ','OQ','NQ')
dropped = [x for x in var_list if x.startswith(prefixes)]

In [215]:
df.drop(dropped, axis = 1 , inplace = True)

In [216]:
var_list = list(df)

In [217]:
retain = [ele for ele in var_list if ele not in unwanted]

In [218]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dimensions = ['unique_id','disecode','student_name','class'
                                  ,'gender','social_category'
                                  ,'English_present_absent'
                                ]

In [219]:
df = pd.melt(df,id_vars = dimensions
                        ,value_vars = retain)

In [220]:
df.head()

,unique_id,disecode,student_name,class,gender,social_category,English_present_absent,variable,value
0,3001029373,28210500904,BESTHA MANOJ,1,B,3,P,Score_NQ1,1.0
1,3000999604,28224800604,ARAVA BHAVISHYA,1,G,3,P,Score_NQ1,1.0
2,3000824764,28230100101,UDAYAGIRI MAHAMMAD ARSHAD,1,B,3,P,Score_NQ1,1.0
3,3000824825,28230100101,C SIFIYAN,1,B,3,P,Score_NQ1,1.0
4,3000824910,28230100101,PAPPUSETTY KULLAYAPPA,1,B,4,P,Score_NQ1,1.0


In [221]:
df[['S','Q']] = df['variable'].str.split('_',1,expand=True)

In [222]:
df[['Subject','Question']] = df['Q'].str.split('Q',1,expand=True)

In [223]:
#Language mapped as Telugu for general files; English and Hindi will be separate scored files altogether
df.loc[df["Subject"]=="L","Subject"]="Telugu"
df.loc[df["Subject"]=="M","Subject"]="Maths"
df.loc[df["Subject"]=="E","Subject"]="EVS"
df.loc[df["Subject"]=="O","Subject"]="SS"
df.loc[df["Subject"]=="S","Subject"]="Science"
df.loc[df["Subject"]=="N","Subject"]="English"

In [224]:
df = df.drop(['variable','S','Q'],axis=1)

In [225]:
df.fillna('',inplace = True)

In [226]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
indexELM = df[(((df['English_present_absent']=='A')|(df['English_present_absent']==''))&(df['Subject']=='English'))
                      ].index

df.drop(indexELM, inplace=True)

In [227]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
df = df.drop(['English_present_absent'],axis=1)

In [228]:
df['Question'] = df['Question'].astype(np.int64)

In [229]:
df = df.rename(columns = {'unique_id':'Child_Id','disecode':'Udise','student_name':'Child_Name'\
                          ,'class':'Grade','gender':'Gender','social_category':'Social_Category'\
                          ,'value':'Score','Subject':'Subject'\
                          ,'Question':'Question_no'})

<h1 style="color:blue;">
    School mapping
</h1>

In [240]:
school_mapping = pd.read_excel('./MappingFiles/AP_SchoolMaster_DistrictMandal_Mapping.xlsx')

In [241]:
school_mapping['SCHOOLCODE'] = school_mapping['SCHOOLCODE'].astype(str)

In [243]:
school_mapping = school_mapping.rename(columns = {'SCHOOLCODE':'Udise'})

In [244]:
df = pd.merge(df,school_mapping,how='left',on='Udise')

In [246]:
question_mapping = pd.read_csv('./MappingFiles/cba1_answer_key_with_question_info.csv')

In [247]:
question_mapping['subject'] =  np.where(question_mapping['subject']=='Language'\
                                        , question_mapping['medium']\
                                        , question_mapping['subject'])

In [248]:
question_mapping['subject'] = np.where(question_mapping['subject']=='Biology','Science'\
                                       ,question_mapping['subject'])

In [249]:
question_mapping['subject'] = np.where(question_mapping['subject']=='Physics','Science'\
                                       ,question_mapping['subject'])

In [250]:
question_mapping = question_mapping[['class','subject','qno','skill_name','topic_name','characteristics']]

In [251]:
question_mapping.drop_duplicates(keep='first', inplace=True)

In [252]:
question_mapping = question_mapping.rename(columns={'class':'Grade','subject':'Subject','qno':'Question_no'})

In [253]:
df = pd.merge(df,question_mapping,how='left',on=['Grade','Subject','Question_no'])

In [254]:
df = df.assign(State='Andhra Pradesh')

In [255]:
df = df.rename(columns={'SCHOOLNAME':'School','schooltype':'School_Type','Udise':'Udise'\
                         ,'Social_Category':'Social_Category','District ':'District'\
                           ,'skill_name':'Skill','topic_name':'Topic','characteristics':'Characteristic'})

In [256]:
df = df[[
'State',
'District',
'Mandal',
'School',
'Udise',
'School_Type',
'Grade',
'Child_Name',
'Child_Id',
'Gender',
'Social_Category',
'Subject',
'Question_no',
'Score',
'Topic',
'Skill',
'Characteristic']]

In [257]:
df.to_pickle('./CBALongPickleFiles/eng.pkl')

In [258]:
#df.to_csv('./CBALongFiles/CBA_G5_LongFile.csv')

In [261]:
df = pd.DataFrame()